In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import cv2


In [ ]:
from cycler import cycler
import matplotlib as mpl

raw_light_palette = [
    (0, 122, 255), # Blue
    (255, 149, 0), # Orange
    (52, 199, 89), # Green
    (255, 59, 48), # Red
    (175, 82, 222),# Purple
    (255, 45, 85), # Pink
    (88, 86, 214), # Indigo
    (90, 200, 250),# Teal
    (255, 204, 0)  # Yellow
]
raw_dark_palette = [
    (10, 132, 255), # Blue
    (255, 159, 10), # Orange
    (48, 209, 88),  # Green
    (255, 69, 58),  # Red
    (191, 90, 242), # Purple
    (94, 92, 230),  # Indigo
    (255, 55, 95),  # Pink
    (100, 210, 255),# Teal
    (255, 214, 10)  # Yellow
]
raw_gray_light_palette = [
    (142, 142, 147),# Gray
    (174, 174, 178),# Gray (2)
    (199, 199, 204),# Gray (3)
    (209, 209, 214),# Gray (4)
    (229, 229, 234),# Gray (5)
    (242, 242, 247),# Gray (6)
]
raw_gray_dark_palette = [
    (142, 142, 147),# Gray
    (99, 99, 102),  # Gray (2)
    (72, 72, 74),   # Gray (3)
    (58, 58, 60),   # Gray (4)
    (44, 44, 46),   # Gray (5)
    (28, 28, 39),   # Gray (6)
]

light_palette = np.array(raw_light_palette)/255
dark_palette = np.array(raw_dark_palette)/255
gray_light_palette = np.array(raw_gray_light_palette)/255
gray_dark_palette = np.array(raw_gray_dark_palette)/255

mpl.rcParams['axes.prop_cycle'] = cycler('color',dark_palette)
mpl.rcParams['figure.facecolor']  = gray_dark_palette[-2]
mpl.rcParams['figure.edgecolor']  = gray_dark_palette[-2]
mpl.rcParams['axes.facecolor'] =  gray_dark_palette[-2]

white_color = gray_light_palette[-2]
mpl.rcParams['text.color'] = white_color
mpl.rcParams['axes.labelcolor'] = white_color
mpl.rcParams['axes.edgecolor'] = white_color
mpl.rcParams['xtick.color'] = white_color
mpl.rcParams['ytick.color'] = white_color

mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False


In [ ]:
# Create a list with the filepaths for training and testing
train_img_Path = '../input/plant-pathology-2021-fgvc8/train_images'

test_img_Path = '../input/plant-pathology-2021-fgvc8/test_images'

img_Path = '../input/resized-plant2021/img_sz_256'
#samples=500, nrows=samples)
train = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/train.csv')

sample_submission = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/sample_submission.csv')


In [ ]:
train.head()

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(14,7))
b = sns.countplot(x='labels', data=train, order=sorted(train['labels'].unique()))
for item in b.get_xticklabels():
    item.set_rotation(90)
plt.title('Label Distribution', weight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(20,40))
i=1
for idx,s in train.head(9).iterrows():
    img_path = os.path.join(img_Path,s['image'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig=plt.subplot(9,3,i)
    fig.imshow(img)
    fig.set_title(s['labels'])
    i+=1

In [ ]:
CLASSES = train['labels'].unique().tolist()
print(CLASSES)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   validation_split=0.25)

train_data = train_datagen.flow_from_dataframe(train,
                                              directory=img_Path,
                                              classes=CLASSES,
                                              x_col="image",
                                              y_col="labels",
                                              target_size=(150, 150),
                                              subset='training')

val_data = train_datagen.flow_from_dataframe(train,
                                            directory=img_Path,
                                            classes=CLASSES,
                                            x_col="image",
                                            y_col="labels",
                                            target_size=(150, 150),
                                            subset='validation')
print(len(CLASSES))

In [ ]:
dict_classes = train_data.class_indices
dict_classes

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.applications import InceptionResNetV2, DenseNet169, ResNet152V2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.metrics import classification_report


In [ ]:
BS=128
EPOCHS=20

#### VGG16

In [ ]:
from tensorflow.keras.layers import Input

base_VGG = VGG16(include_top = False, 
                weights = 'imagenet', 
                input_shape = train_data.image_shape, 
                pooling='avg',
                classes = CLASSES)

#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_VGG = Sequential()
model_VGG.add(base_VGG)
model_VGG.add(Dense(12, activation=('softmax')))
model_VGG.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_VGG.summary()


# Training the CNN on the Train data and evaluating it on the val data
# b = model_VGG.fit(train_data, validation_data = val_data, epochs = EPOCHS, batch_size=BS)
H_VGG = model_VGG.fit(train_data, validation_data = val_data, epochs = EPOCHS, batch_size=BS)

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_VGG.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_VGG.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H_VGG.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H_VGG.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
print(len(val_data.labels))
print(len(dict_classes))
predIdxs_VGG = model_VGG.predict(val_data, batch_size=BS)
print(len(predIdxs_VGG))


In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs_VGG = model_VGG.predict(val_data, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs_VGG = np.argmax(predIdxs_VGG, axis=1)
# show a nicely formatted classification report
print(classification_report(val_data.labels, predIdxs_VGG, labels=[0,1, 2, 3,4,5,6,7,8,9,10,11]))


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print ("f1_score",metrics.f1_score(val_data.labels, predIdxs_VGG, average='weighted', labels=np.unique(predIdxs_VGG)))
print ("accuracy",accuracy_score(val_data.labels, predIdxs_VGG))
print ("recall_score",recall_score(val_data.labels, predIdxs_VGG, average='micro'))
print ("precision_score",precision_score(val_data.labels, predIdxs_VGG, average='micro'))


#### ResNet152V2 

In [ ]:
base_Net = ResNet152V2(include_top = False, 
                         weights = '../input/keras-pretrained-models/ResNet152V2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)


In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_ResNet = Sequential()
model_ResNet.add(base_Net)
model_ResNet.add(Dense(12, activation=('softmax')))

model_ResNet.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_ResNet.summary()

# Training the CNN on the Train data and evaluating it on the val data
H_ResNet152V2 = model_ResNet.fit(train_data, validation_data = val_data, epochs = EPOCHS, batch_size=BS)

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_ResNet152V2.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_ResNet152V2.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H_ResNet152V2.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H_ResNet152V2.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
# make predictions on the testing set
predIdxs_ResNet = model_ResNet.predict(val_data, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs_ResNet = np.argmax(predIdxs_ResNet, axis=1)
# show a nicely formatted classification report
#we can use the target name nested of labels but it will give error because of sizes or labels=np.unique(predIdxs_VGG) buth appear onlt non zero classes
print(classification_report(val_data.labels, predIdxs_ResNet, labels=[0,1, 2, 3,4,5,6,7,8,9,10,11]))



In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print ("f1_score",metrics.f1_score(val_data.labels, predIdxs_VGG, average='weighted', labels=np.unique(predIdxs_VGG)))
print ("accuracy",accuracy_score(val_data.labels, predIdxs_VGG))
print ("recall_score",recall_score(val_data.labels, predIdxs_VGG, average='micro'))
print ("precision_score",precision_score(val_data.labels, predIdxs_VGG, average='micro'))


####  InceptionResNetV2

In [ ]:
base_InceptionResNetV2 = InceptionResNetV2(include_top = False, 
                         weights = '../input/keras-pretrained-models/InceptionResNetV2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)


In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_INResNet2 = Sequential()
model_INResNet2.add(base_InceptionResNetV2)
model_INResNet2.add(Dense(12, activation=('softmax')))

model_INResNet2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_INResNet2.summary()

# Training the CNN on the Train data and evaluating it on the val data
H_INResNet = model_INResNet2.fit(train_data, validation_data = val_data, epochs = EPOCHS, batch_size=BS)


In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_INResNet.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_INResNet.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H_INResNet.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H_INResNet.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs_INResNet2 = model_INResNet2.predict(val_data, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs_INResNet2 = np.argmax(predIdxs_INResNet2, axis=1)
# show a nicely formatted classification report
print(classification_report(val_data.labels, predIdxs_INResNet2, labels=[0,1, 2, 3,4,5,6,7,8,9,10,11]))
##


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print ("f1_score",metrics.f1_score(val_data.labels, predIdxs_VGG, average='weighted', labels=np.unique(predIdxs_VGG)))
print ("accuracy",accuracy_score(val_data.labels, predIdxs_VGG))
print ("recall_score",recall_score(val_data.labels, predIdxs_VGG, average='micro'))
print ("precision_score",precision_score(val_data.labels, predIdxs_VGG, average='micro'))


####  DenseNet169 

In [ ]:
base_DenseNet169 = DenseNet169(include_top = False, 
                         weights = '../input/keras-pretrained-models/DenseNet169_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)


In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_dense = Sequential()
model_dense.add(base_DenseNet169)
model_dense.add(Dense(12, activation=('softmax')))

model_dense.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_dense.summary()

# Training the CNN on the Train data and evaluating it on the val data
H_Dense = model_dense.fit(train_data, validation_data = val_data, epochs = EPOCHS, batch_size=BS)


In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_Dense.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_Dense.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H_Dense.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H_Dense.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs_dense = model_dense.predict(val_data, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs_dense = np.argmax(predIdxs_dense, axis=1)
# show a nicely formatted classification report
print(classification_report(val_data.labels, predIdxs_dense, labels=[0,1, 2, 3,4,5,6,7,8,9,10,11]))
##


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print ("f1_score",metrics.f1_score(val_data.labels, predIdxs_VGG, average='weighted', labels=np.unique(predIdxs_VGG)))
print ("accuracy",accuracy_score(val_data.labels, predIdxs_VGG))
print ("recall_score",recall_score(val_data.labels, predIdxs_VGG, average='micro'))
print ("precision_score",precision_score(val_data.labels, predIdxs_VGG, average='micro'))
